# A6: predictions demo

# Introduction

This notebook describes the COBRA methods for flux prediction in jQMM: MoMA (Minimization of Metabolic Adjustment) and ROOM (Regulatory On/Off Minimization), which were used in Garcia Martin *et al* 2015. 

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

And importing needed modules:

In [3]:
import predictions, FluxModels,core
import copy

In [4]:
cd ~/tests

/mnt/nfs.jbei/nfs/users/hgmartin/tests


# Results

The main method in this modules is the function *predict*, which is used to find flux predictions through the MoMA or ROOM methods.

In order to demonstrate it, let's first calculate the base flux profiles (i.e. flux profile before KO):

In [5]:
FBAfileName = os.environ["QUANTMODELPATH"]+'/data/tests/Toya2010/2S/2SpaperTest/XML/'+'EciJR904wt5hGrowth'+'Partial'+'.xml'    
FBAmodel    = FluxModels.FBAModel(FBAfileName)

In [6]:
FBAresults   = FBAmodel.findFluxes()

We need now a base reaction network holding the upper and lower bounds of the predicted network:

In [7]:
ReacNet   = copy.deepcopy(FBAmodel.reactionNetwork)

# Change flux bounds to avoid biasing prediction
ReacNet.changeFluxBounds('GLCpts'      ,core.fluxBounds( 0, 25  ,False)[1])      
ReacNet.changeFluxBounds('EX_glc_e_'   ,core.fluxBounds(-15,-0  ,True,True)[1])
ReacNet.changeFluxBounds('BiomassEcoli' ,core.fluxBounds( 0, 25  ,False)[1])      
ReacNet.changeFluxBounds('EX_ac_e_'    ,core.fluxBounds( 0, 25  ,True,True)[1])

Let's now calculate the predicted fluxes if the reaction 'PGI' is knocked out using MOMA:

In [8]:
MOMAresults    = predictions.predict(FBAresults,'PGI','MOMA', ReacNet.getSBMLString())

and ROOM:

In [9]:
ROOMresults    = predictions.predict(FBAresults,'PGI','ROOM', ReacNet.getSBMLString())

In [10]:
MOMAdict = MOMAresults.reactionNetwork.reactionList.getFluxDictionary()

In [11]:
ROOMdict = ROOMresults.reactionNetwork.reactionList.getFluxDictionary()

Predictions of PGI flux for both method are the same (since it corresponds to the reaction being eliminated):

In [12]:
print MOMAdict['PGI'].net
print ROOMdict['PGI'].net

0.0
0.0


But the MoMA and ROOM methods provide different predictions for other fluxes:

In [13]:
print MOMAdict['G6PDH2r'].net
print ROOMdict['G6PDH2r'].net

9.89938763494
14.6158706172


# References

Martín, Héctor García, et al. "A method to constrain genome-scale models with 13 C labeling data." PLoS Comput Biol 11.9 (2015): e1004363.
